# 调参指引
> 目标：解释并推导使用 FlyEval 输出的参数 + PDF 给出的动力学/推进器模型，就能在悬停工作点附近得到一个标准线性被控对象，并用极点配置直接解出 PID 初值。

> 网站：https://www.flyeval.com/index.html

> https://www.flyeval.com/js/MathModelDocEn.pdf


## 步骤 0： 准备工作

### FlyEval 输入参数并得到：

- 质量：$m$
- 转动惯量：$J_{xx},J_{yy},J_{zz}$
- 臂长：$d$（质心到电机轴线距离）
- 推力/反扭矩系数：$C_t, C_m$
- 油门到稳态转速线性映射：$\omega_{ss}=C_R\sigma+\omega_b$
- 电机一阶时间常数：$T_m$



### PDF 给出的“理论基础”（设计 PID 必须用到的四块）

> 1. **推力/力矩与转速**  
   $$T=C_t\omega^2,\qquad M=C_m\omega^2.$$

> 2. **电机/推进器动态**  
   $$\omega(s)=\frac{\omega_{ss}(s)}{T_m s+1},\qquad \omega_{ss}=C_R\sigma+\omega_b.$$

> 3. **刚体姿态动力学**  
   $$\mathbf{J}\dot{\boldsymbol{\omega}} = -\boldsymbol{\omega}\times(\mathbf{J}\boldsymbol{\omega})+\boldsymbol{\tau}+\cdots.$$  
   在悬停小角速度附近可近似为  
   $$\mathbf{J}\dot{\boldsymbol{\omega}}\approx \boldsymbol{\tau}.$$

> 4. **控制分配（mixer）**  
   $$\begin{bmatrix} f & \tau_x & \tau_y & \tau_z\end{bmatrix}^\top
   =
   \mathbf{A}\begin{bmatrix}\omega_1^2 & \cdots & \omega_n^2\end{bmatrix}^\top,$$
   并对四旋 X/+/六旋等给出矩阵。

---

## 步骤 1： 控制理论推导

### 关键思想：在悬停点把非线性系统变成“近似线性系统”。把系统化成一个标准结构，然后用标准方法把闭环极点放到你想要的位置。


多旋翼真实模型是非线性的（$\omega^2$、$\sin\theta$、$\omega\times J\omega$ 等）。但控制工程里有个最核心的套路：

> **在实际工作点（悬停）附近，小扰动行为可以用线性系统近似。**

这一步就是“线性化”（在工作点做泰勒展开并取一阶项）。线性系统的好处是：你能用传递函数、极点配置、频域指标直接反推出 PID。

### 1.2 你最终会得到一个“通用被控对象”

对最常见的 **角速度 Rate 环**（roll/pitch/yaw 的 $p,q,r$）：

- 控制器输出的是某个轴向的“差分油门命令”（本质：让某些电机加速、另一些减速）
- 电机转速经过一阶惯性 $T_m$
- 推力/反扭矩由 $\omega^2$ 产生（线性化后与 $\delta\omega$ 成正比）
- 刚体角速度对力矩积分（因为 $J\dot{\omega}=\tau$）

因此结构一定是：

$$
\boxed{
\frac{\Omega_{\text{axis}}(s)}{u(s)}
= \frac{K}{s(T_m s+1)}
}
$$

- **一个积分环节 $1/s$**：角速度是角加速度积分来的；
- **一个一阶滞后 $1/(T_m s+1)$**：电机不可能瞬时改变转速。

PID（尤其带 D）对“积分 + 一阶滞后”这种对象非常对口：

- P：提供主要刚度（响应快慢）
- I：消除稳态误差（抗扰后回到 0 偏差）
- D：提供相位超前（抑制振荡、提高阻尼）

---


## 2. 从 PDF 方程出发，完整推导被控对象 $\;K/(s(T_m s+1))$

### Step A：先找悬停工作点 $\omega_0$

单桨推力：
$$
T=C_t\omega^2
$$

四旋（或 $n$ 旋翼）悬停时各桨同速 $\omega_0$，总推力 $f=nC_t\omega_0^2$。悬停力平衡 $f=mg$，得：

$$
\boxed{
\omega_0=\sqrt{\frac{mg}{nC_t}}
}
$$

---

### Step B：油门小扰动 $\delta\sigma$ 如何引起转速小扰动 $\delta\omega$

PDF 给：
$$
\omega_{ss}=C_R\sigma+\omega_b,\qquad 
\omega(s)=\frac{\omega_{ss}(s)}{T_m s+1}
$$

对悬停点做小扰动（$\sigma=\sigma_0+\delta\sigma,\ \omega=\omega_0+\delta\omega$），得到小信号关系：

$$
\delta\omega(s)
=\frac{C_R}{T_m s+1}\delta\sigma(s)
$$

---

### Step C：$\delta\omega$ 如何引起推力/力矩小扰动（关键线性化）

推力是非线性 $\omega^2$。在 $\omega_0$ 处一阶线性化：

$$
T(\omega)=C_t\omega^2
\Rightarrow
\delta T \approx \left.\frac{dT}{d\omega}\right|_{\omega_0}\delta\omega
=2C_t\omega_0\,\delta\omega
$$

同理反扭矩：
$$
M(\omega)=C_m\omega^2
\Rightarrow
\delta M \approx 2C_m\omega_0\,\delta\omega
$$

把 Step B 代入：
$$
\delta T(s)=\frac{2C_t\omega_0C_R}{T_m s+1}\delta\sigma(s),\qquad
\delta M(s)=\frac{2C_m\omega_0C_R}{T_m s+1}\delta\sigma(s)
$$

---

### Step D：用控制分配（mixer）把电机小信号变成轴向力矩 $\delta\tau$

PDF 给了控制分配：
$$
\begin{bmatrix} f & \tau_x & \tau_y & \tau_z\end{bmatrix}^\top
=
\mathbf{A}
\begin{bmatrix}\omega_1^2 & \cdots & \omega_n^2\end{bmatrix}^\top
$$

对四旋 **X** 构型，roll 的典型差分方式是“两台增加、两台减少、幅值相等”（使一阶总推力不变，但产生滚转力矩）。  
把这个差分模式代入分配矩阵，会得到 roll 力矩与 $\delta(\omega^2)$ 成正比（下面给出常用幅值关系）：

- 由于 $\delta(\omega^2)\approx 2\omega_0\delta\omega$，且 $\delta\omega$ 与 $\delta\sigma$ 经过电机一阶环节，
- roll/pitch 主要由推力差产生力矩（用 $C_t$），yaw 由反扭矩差产生（用 $C_m$）。

常用的小信号“力矩有效度”（取幅值）写成：

**Roll / Pitch：**
$$
\delta\tau_{rp}(s)
=
\frac{4\sqrt{2}\,d\,C_t\,\omega_0\,C_R}{T_m s+1}\ \delta\sigma_{rp}(s)
$$

**Yaw（幅值）：**
$$
|\delta\tau_{z}(s)|
=
\frac{8\,C_m\,\omega_0\,C_R}{T_m s+1}\ |\delta\sigma_{yaw}(s)|
$$

> 注：具体系数（比如 $4\sqrt{2}$）依赖你对 X 构型臂长定义、坐标轴、以及差分命令的归一化方式；只要你用同一套 mixer 定义推导，形式结构必然是“电机一阶 + 常数增益”。

---

### Step E：刚体动力学把 $\delta\tau$ 变成角速度响应

PDF 姿态动力学：
$$
\mathbf{J}\dot{\boldsymbol{\omega}}
=
-\boldsymbol{\omega}\times(\mathbf{J}\boldsymbol{\omega})
+\boldsymbol{\tau}+\cdots
$$

悬停附近：角速度小，耦合项 $-\omega\times(J\omega)$ 为二阶小量，先忽略：

$$
\mathbf{J}\dot{\boldsymbol{\omega}}\approx \boldsymbol{\tau}
$$

以 roll 轴为例：
$$
\dot{p}\approx \frac{\tau_x}{J_{xx}}
\Rightarrow
\frac{p(s)}{\tau_x(s)}=\frac{1}{J_{xx}s}
$$

把 Step D 的 $\tau/u$ 合并，得到通用形式：

$$
\boxed{
\frac{p(s)}{u_{\text{roll}}(s)}
= \frac{K_{\text{roll}}}{s(T_m s+1)}
}
$$

其中（按上面的归一化）：
$$
\boxed{
K_{\text{roll}}
=\frac{4\sqrt2\,d\,C_t\,\omega_0\,C_R}{J_{xx}}
}
$$

同理：
$$
K_{\text{pitch}}=\frac{4\sqrt2\,d\,C_t\,\omega_0\,C_R}{J_{yy}},\qquad
K_{\text{yaw}}=\frac{8\,C_m\,\omega_0\,C_R}{J_{zz}}
$$

> 注：Yaw 的符号取决于电机旋向与 mixer 约定；工程上通常先用幅值算增益，再在分配矩阵/符号约定里统一处理正负。

---

## 3. 用“极点配置”推 PID 初值（控制理论最干净的一步）

现在你的被控对象是：

$$
G(s)=\frac{K}{s(T_m s+1)}
$$

选 PID：
$$
C(s)=K_p+\frac{K_i}{s}+K_d s
$$

闭环特征方程来自 $1+C(s)G(s)=0$，展开得到：

$$
T_m s^3+(1+K K_d)s^2+(K K_p)s+(K K_i)=0
$$

含义：

- 这是一个三阶系统（积分 + 电机一阶 + 控制器引入的零点/系数）
- 设计 PID 本质：**让三阶多项式的根（闭环极点）落到你想要的位置**

---

### 3.1 选你想要的“响应品质”（新手也能用）

常用目标：**二阶主导极点 + 一个更快的附加极点**：

$$
(s^2+2\zeta\omega_n s+\omega_n^2)(s+p_3)=0
$$

- $\zeta$：阻尼比（0.6–0.9 常用；越大越不振荡）
- $\omega_n$：自然频率（决定快慢；与带宽同量级）
- $p_3$：第三极点（常取 $p_3=\alpha\omega_n,\ \alpha=3\sim 6$，让它更快不抢主导）

---

### 3.2 系数对齐，直接解出 PID（“初值公式”）

把目标多项式展开并乘上 $T_m$（使最高次项系数对齐），逐项对齐可得：

$$
\boxed{
\begin{aligned}
K_d &= \frac{T_m(2\zeta\omega_n+p_3)-1}{K}\\[4pt]
K_p &= \frac{T_m(\omega_n^2+2\zeta\omega_n p_3)}{K}\\[4pt]
K_i &= \frac{T_m(\omega_n^2 p_3)}{K}
\end{aligned}}
$$

这三条就是你要的“控制理论依据”链路：

**PDF 模型线性化 → 标准对象 $K/(s(T_m s+1))$ → 选择闭环极点 → 系数匹配得到 PID。**

---



## 4. 新手怎么选 $\zeta,\ \omega_n,\ p_3$

### 4.1 $\zeta$（阻尼比）

- 想稳、不想震：$\zeta=0.7$（经典折中）
- 想更“肉”、更抗噪：$\zeta=0.9$
- 不建议上来用 $<0.5$（更容易振荡）

### 4.2 $\omega_n$（快慢）

用两条简单规则定数量级：

1. **别超过控制频率的 1/10**  
   如果 rate 环 250 Hz，带宽别超过约 25 Hz（留余量）

2. **roll/pitch 通常比 yaw 快**  
   例如：roll/pitch 4–8 Hz；yaw 2–4 Hz（初值阶段）

换算：$\omega_n = 2\pi f$（rad/s）

### 4.3 $p_3$（第三极点）

默认取：
$$
p_3=\alpha\omega_n,\qquad \alpha=4
$$

含义：第三极点更快，让三阶系统“规整”，但不显著改变主二阶形状。

---

## 5. Python 脚本：从 FlyEval 参数算 PID（连续域初值）

> 这是**连续域 PID 初值**。上飞控前还需要：离散化、限幅、抗积分饱和、滤波/延迟处理等。


In [3]:
import math
from dataclasses import dataclass

@dataclass
class Params:
    # FlyEval / PDF parameters (assumed rad/s-based)
    m: float
    g: float
    Jxx: float
    Jyy: float
    Jzz: float
    d: float
    Ct: float       # N/(rad/s)^2
    Cm: float       # N*m/(rad/s)^2
    CR: float       # (rad/s) per throttle
    omega_b: float  # rad/s (not used in small-signal gain, kept for completeness)
    Tm: float       # seconds
    n_rotors: int = 4

@dataclass
class Spec:
    zeta: float = 0.7
    f_roll_hz: float = 6.0
    f_pitch_hz: float = 6.0
    f_yaw_hz: float = 3.0
    alpha: float = 4.0  # p3 = alpha * wn

def _check_params(p: Params):
    for name in ["m","g","Jxx","Jyy","Jzz","d","Ct","Cm","CR","Tm","n_rotors"]:
        val = getattr(p, name)
        if val is None:
            raise ValueError(f"{name} is None")
    if p.m <= 0: raise ValueError("m must be > 0")
    if p.g <= 0: raise ValueError("g must be > 0")
    if p.Jxx <= 0 or p.Jyy <= 0 or p.Jzz <= 0: raise ValueError("Jxx/Jyy/Jzz must be > 0")
    if p.d <= 0: raise ValueError("d must be > 0")
    if p.Ct <= 0: raise ValueError("Ct must be > 0")
    if p.Cm <= 0: raise ValueError("Cm must be > 0")
    if p.CR == 0: raise ValueError("CR must be non-zero")
    if p.Tm <= 0: raise ValueError("Tm must be > 0")
    if p.n_rotors < 3: raise ValueError("n_rotors must be >= 3")

def hover_omega0(p: Params) -> float:
    # omega0 = sqrt(mg/(n Ct))
    return math.sqrt(p.m * p.g / (p.n_rotors * p.Ct))

def pid_from_pole_match(K: float, Tm: float, wn: float, zeta: float, alpha: float):
    """
    Plant: omega/u = K / (s (Tm s + 1))
    Target: (s^2 + 2*zeta*wn*s + wn^2)(s + p3), p3 = alpha*wn
    """
    if wn <= 0: raise ValueError("wn must be > 0")
    if not (0 < zeta <= 2.0): raise ValueError("zeta out of reasonable range")
    if alpha <= 0: raise ValueError("alpha must be > 0")
    if K == 0: raise ValueError("K must be non-zero")

    p3 = alpha * wn
    Kd = (Tm * (2*zeta*wn + p3) - 1.0) / K
    Kp = (Tm * (wn**2 + 2*zeta*wn*p3)) / K
    Ki = (Tm * (wn**2 * p3)) / K
    return Kp, Ki, Kd

def compute_quad_x_rate_pid(p: Params, s: Spec, yaw_use_abs: bool = True):
    """
    QUAD X small-signal effectiveness (magnitude) used:
      tau_x/u = 4*sqrt(2)*d*Ct*omega0*CR / (Tm s + 1)
      tau_y/u same
      tau_z/u = 8*Cm*omega0*CR / (Tm s + 1)  (sign depends on convention)
    rate dynamics: J * omega_dot = tau  => omega/tau = 1/(J s)
    """
    _check_params(p)

    w0 = hover_omega0(p)

    # torque per unit differential throttle (N*m per u)
    k_tau_rp = 4.0 * math.sqrt(2.0) * p.d * p.Ct * w0 * p.CR
    k_tau_yaw = 8.0 * p.Cm * w0 * p.CR

    # rate plant gains K (rad/s^2 per u)
    K_roll  = k_tau_rp / p.Jxx
    K_pitch = k_tau_rp / p.Jyy
    K_yaw   = k_tau_yaw / p.Jzz

    wn_roll  = 2*math.pi*s.f_roll_hz
    wn_pitch = 2*math.pi*s.f_pitch_hz
    wn_yaw   = 2*math.pi*s.f_yaw_hz

    pid_roll  = pid_from_pole_match(K_roll, p.Tm, wn_roll, s.zeta, s.alpha)
    pid_pitch = pid_from_pole_match(K_pitch, p.Tm, wn_pitch, s.zeta, s.alpha)

    if yaw_use_abs:
        pid_yaw = pid_from_pole_match(abs(K_yaw), p.Tm, wn_yaw, s.zeta, s.alpha)
    else:
        pid_yaw = pid_from_pole_match(K_yaw, p.Tm, wn_yaw, s.zeta, s.alpha)

    # quick sanity: warn if Kd < 0
    def _warn(name, pid):
        Kp, Ki, Kd = pid
        if Kd < 0:
            print(f"[WARN] {name}: Kd is negative ({Kd:.4g}). "
                  f"Try increasing f_*_hz or alpha, or check units (Ct/CR).")

    _warn("roll", pid_roll)
    _warn("pitch", pid_pitch)
    _warn("yaw", pid_yaw)

    return {
        "hover_omega0_rad_s": w0,
        "K_roll": K_roll,
        "K_pitch": K_pitch,
        "K_yaw_raw": K_yaw,
        "PID_rate_roll (Kp, Ki, Kd)": pid_roll,
        "PID_rate_pitch (Kp, Ki, Kd)": pid_pitch,
        "PID_rate_yaw (Kp, Ki, Kd)": pid_yaw,
        "notes": [
            "Assumes rad/s-based Ct,Cm,CR. Convert from rpm-based data if needed.",
            "Yaw sign depends on your mixer and motor spin directions; yaw_use_abs=True returns magnitude-only PID.",
            "These are continuous-time gains for the plant omega/u = K / (s(Tm s + 1))."
        ]
    }

# ---- Example run (replace with your FlyEval outputs) ----
p = Params(
    m=2.50, g=9.81,
    Jxx=0.035, Jyy=0.040, Jzz=0.070,
    d=0.215,
    Ct=1.8e-5, Cm=2.0e-7,
    CR=900.0, omega_b=0.0,
    Tm=0.03,
    n_rotors=4
)

s = Spec(zeta=0.7, f_roll_hz=6.0, f_pitch_hz=6.0, f_yaw_hz=3.0, alpha=4.0)

out = compute_quad_x_rate_pid(p, s, yaw_use_abs=True)
out


{'hover_omega0_rad_s': 583.6308764964376,
 'K_roll': 328.5478874796595,
 'K_pitch': 287.47940154470206,
 'K_yaw_raw': 12.006120887926715,
 'PID_rate_roll (Kp, Ki, Kd)': (0.8565027242833253,
  19.569328482822826,
  0.015544936714574827),
 'PID_rate_pitch (Kp, Ki, Kd)': (0.9788602563238004,
  22.364946837511802,
  0.01776564195951409),
 'PID_rate_yaw (Kp, Ki, Kd)': (5.8595562070101295,
  66.93941357541634,
  0.17104842425453132),
 'notes': ['Assumes rad/s-based Ct,Cm,CR. Convert from rpm-based data if needed.',
  'Yaw sign depends on your mixer and motor spin directions; yaw_use_abs=True returns magnitude-only PID.',
  'These are continuous-time gains for the plant omega/u = K / (s(Tm s + 1)).']}

Autotune官方文档地址：https://docs.px4.io/main/en/config/autotune_mc